# Generate synthetic data

In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

## Register cell annotations to H&E

`Xenium_FFPE_Human_Breast_Cancer_Rep1_cells.csv` and `Cell_Barcode_Type_Matrices.csv` available in [https://www.10xgenomics.com/products/xenium-in-situ/preview-dataset-human-breast](https://www.10xgenomics.com/products/xenium-in-situ/preview-dataset-human-breast)

In [ ]:
cells = pd.read_csv('Xenium_FFPE_Human_Breast_Cancer_Rep1_cells.csv')
annotations = pd.read_excel('Cell_Barcode_Type_Matrices.xlsx', sheet_name='Xenium R1 Fig1-5 (supervised)')

In [ ]:
cell_annotations = pd.merge(cells, annotations, how='left', left_on=['cell_id'], right_on=['Barcode']).dropna()
cell_annotations.x_centroid*=4.70588
cell_annotations.y_centroid*=4.70953

In [ ]:
cell_annotations['x_temp'] = cell_annotations.x_centroid//6
cell_annotations['y_temp'] = cell_annotations.y_centroid//6

cell_annotations.y_temp = cell_annotations.y_temp.max() - cell_annotations.y_temp

affine_matrix = np.array([[1.711258423423306, 0.006516026161167, -1131.690570250315], 
                          [-0.009489031327157, 1.710461398370734, -1140.7384447381096],
                          [0, 0, 1]])


coords =  np.matmul(np.linalg.inv(affine_matrix),
                    np.array([cell_annotations.x_temp, 
                              cell_annotations.y_temp, 
                              np.ones(len(cell_annotations))])).transpose()

cell_annotations['x_he'] = coords[:,0]
cell_annotations['y_he'] = coords[:,1]

cell_annotations['x_he']*=6
cell_annotations['y_he']*=6

In [ ]:
cell_annotations.to_csv('cell_annotations.csv')

## Register cell annotations to QuPath detections

This step requires detecting cells using QuPath and saving as `measurements.csv`

In [ ]:
measurements = pd.read_csv('measurements.csv')
measurements['x_centroid']=measurements['Centroid X µm']/0.3528
measurements['y_centroid']=measurements['Centroid Y µm']/0.3528

cell_annotations = pd.read_csv('cell_annotations.csv')
cell_annotations=cell_annotations[['x_he','y_he','Cluster']]
cell_annotations=cell_annotations[cell_annotations.Cluster!='Unlabeled']

In [ ]:
distances = cdist(measurements[['x_centroid', 'y_centroid']], cell_annotations[['x_he', 'y_he']])
closest_clusters = cell_annotations.iloc[distances.argmin(axis=1)]['Cluster'].tolist()
measurements['Assigned_Cluster'] = closest_clusters

In [ ]:
measurements.to_csv('annotated_measurements.csv')

## Generate synthetic Visium spots

In [ ]:
df = pd.read_csv('annotated_measurements.csv')
df['x_centroid']*=0.3528
df['y_centroid']*=0.3528

In [ ]:
# Extract the range of x and y coordinates
x_min, x_max = df['x_centroid'].min(), df['x_centroid'].max()
y_min, y_max = df['y_centroid'].min(), df['y_centroid'].max()

# Calculate the number of hexagons in x and y directions
num_hexagons_x = int((x_max - x_min) / (np.sqrt(3) * 50))
num_hexagons_y = int((y_max - y_min) / (2 * 50))

# Function to generate hexagonal grid points
def generate_hexagonal_grid():
    hexagon_centers = []
    for i in range(num_hexagons_x + 1):
        for j in range(num_hexagons_y + 1):
            x_offset = x_min + i * np.sqrt(3) * 50
            y_offset = y_min + j * 2 * 50 + (i % 2) * 50
            hexagon_centers.append((x_offset, y_offset))
    return hexagon_centers

# Generate hexagonal grid points
hexagon_centers = generate_hexagonal_grid()

# Create DataFrame for visium spots
df_visium_spots = pd.DataFrame(hexagon_centers, columns=['x_visium', 'y_visium'])

# Filter out points outside the rectangle defined by x_centroid and y_centroid
df_visium_spots = df_visium_spots[(df_visium_spots['x_visium'] >= x_min) & (df_visium_spots['x_visium'] <= x_max)
                                & (df_visium_spots['y_visium'] >= y_min) & (df_visium_spots['y_visium'] <= y_max)]

# Plotting the original centroids and visium spots as circles
fig, ax = plt.subplots()
ax.scatter(df['x_centroid'], df['y_centroid'], marker='.', color='blue', label='Original Centroids')

for index, row in df_visium_spots.iterrows():
    ellipse = Ellipse(xy=(row['x_visium'], row['y_visium']), width=55, height=55, edgecolor='red', facecolor='red', alpha=0.5)
    ax.add_patch(ellipse)

ax.set_xlabel('x_centroid')
ax.set_ylabel('y_centroid')
ax.legend()
plt.show()

In [ ]:
df_visium_spots.to_csv('visium_spots.csv', index=False)

## Keep cells inside spots

In [ ]:
annotated_measurements = pd.read_csv('annotated_measurements.csv')
visium_spots = pd.read_csv('visium_spots.csv')

In [ ]:
filtered_visium_spots = visium_spots.copy()

for index, row in visium_spots.iterrows():
    x_visium, y_visium = row['x_visium'], row['y_visium']
    
    if (
        (x_visium - 55/2 >= annotated_measurements['Centroid X µm'].min()) and
        (x_visium + 55/2 <= annotated_measurements['Centroid X µm'].max()) and
        (y_visium - 55/2 >= annotated_measurements['Centroid Y µm'].min()) and
        (y_visium + 55/2 <= annotated_measurements['Centroid Y µm'].max())
    ):
        continue
    else:
        filtered_visium_spots = filtered_visium_spots.drop(index)

filtered_annotated_measurements = annotated_measurements.copy()

for index, row in annotated_measurements.iterrows():
    x_centroid, y_centroid = row['Centroid X µm'], row['Centroid Y µm']
    
    if any(
        ((x_centroid - x_visium)**2 + (y_centroid - y_visium)**2)**0.5 <= 55/2
        for x_visium, y_visium in zip(filtered_visium_spots['x_visium'], filtered_visium_spots['y_visium'])
    ):
        continue
    else:
        filtered_annotated_measurements = filtered_annotated_measurements.drop(index)

In [ ]:
filtered_annotated_measurements['n_spot'] = None

for index, row in filtered_annotated_measurements.iterrows():
    x_centroid, y_centroid = row['Centroid X µm'], row['Centroid Y µm']
    
    matching_spot = filtered_visium_spots[
        ((filtered_visium_spots['x_visium'] - x_centroid)**2 + (filtered_visium_spots['y_visium'] - y_centroid)**2)**0.5 <= 55/2
    ]
    
    filtered_annotated_measurements.at[index, 'n_spot'] = matching_spot.index.values[0] if not matching_spot.empty else None

In [ ]:
filtered_annotated_measurements.to_csv('filtered_annotated_measurements.csv')
filtered_visium_spots.to_csv('filtered_visium_spots.csv')